# Many Models Forecasting (MMF)
This notebook showcases how to run MMF with local models on multiple univariate time series of hourly resolution. We will use [M4 competition](https://www.sciencedirect.com/science/article/pii/S0169207019301128#sec5) data.

### Cluster setup

We recommend using a cluster with [Databricks Runtime 16.4 LTS for ML](https://docs.databricks.com/en/release-notes/runtime/16.4lts-ml.html). The cluster can be either a single-node or multi-node CPU cluster. MMF leverages [Pandas UDF](https://docs.databricks.com/en/udf/pandas.html) under the hood and utilizes all the available resource. Make sure to set the following Spark configurations before you start your cluster: [`spark.sql.execution.arrow.enabled true`](https://spark.apache.org/docs/3.0.1/sql-pyspark-pandas-with-arrow.html#enabling-for-conversion-tofrom-pandas) and [`spark.sql.adaptive.enabled false`](https://spark.apache.org/docs/latest/sql-performance-tuning.html#adaptive-query-execution). You can do this by specifying [Spark configuration](https://docs.databricks.com/en/compute/configure.html#spark-configuration) in the advanced options on the cluster creation page.

### Install and import packages
Check out [requirements.txt](https://github.com/databricks-industry-solutions/many-model-forecasting/blob/main/requirements.txt) if you're interested in the libraries we use.

In [0]:
%pip install -r ../../requirements.txt --quiet
dbutils.library.restartPython()

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
import logging
logger = spark._jvm.org.apache.log4j
logging.getLogger("py4j.java_gateway").setLevel(logging.ERROR)
logging.getLogger("py4j.clientserver").setLevel(logging.ERROR)

In [0]:
import pathlib
import pandas as pd
from datasetsforecast.m4 import M4
from mmf_sa import run_forecast

### Prepare data 
We are using [`datasetsforecast`](https://github.com/Nixtla/datasetsforecast/tree/main/) package to download M4 data. M4 dataset contains a set of time series which we use for testing MMF. Below we have written a number of custome functions to convert M4 time series to an expected format.

In [0]:
# Number of time series
n = 1000


def create_m4_hourly():
    y_df, _, _ = M4.load(directory=str(pathlib.Path.home()), group="Hourly")
    _ids = [f"H{i}" for i in range(1, n)]
    y_df = (
        y_df.groupby("unique_id")
        .filter(lambda x: x.unique_id.iloc[0] in _ids)
        .groupby("unique_id")
        .apply(transform_group)
        .reset_index(drop=True)
    )
    return y_df


def transform_group(df):
    unique_id = df.unique_id.iloc[0]
    if len(df) > 720:
        df = df.iloc[-720:]
    _start = pd.Timestamp("2025-01-01 00:00")
    _end = _start + pd.DateOffset(hours=len(df)-1)
    date_idx = pd.date_range(start=_start, end=_end, freq="H", name="ds")
    res_df = pd.DataFrame(data=[], index=date_idx).reset_index()
    res_df["unique_id"] = unique_id
    res_df["y"] = df.y.values
    return res_df

We are going to save this data in a delta lake table. Provide catalog and database names where you want to store the data.

In [0]:
catalog = "mmf" # Name of the catalog we use to manage our assets
db = "m4" # Name of the schema we use to manage our assets (e.g. datasets)
user = spark.sql('select current_user() as user').collect()[0]['user'] # User email address

In [0]:
# Making sure that the catalog and the schema exist
_ = spark.sql(f"CREATE CATALOG IF NOT EXISTS {catalog}")
_ = spark.sql(f"CREATE SCHEMA IF NOT EXISTS {catalog}.{db}")

(
    spark.createDataFrame(create_m4_hourly())
    .write.format("delta").mode("overwrite")
    .saveAsTable(f"{catalog}.{db}.m4_hourly_train")
)

2.35MiB [00:00, 59.5MiB/s]                 
ERROR:datasetsforecast.utils:ERROR, something went wrong downloading data
INFO:datasetsforecast.utils:Successfully downloaded Hourly-train.csv, 2347115, bytes.
133kiB [00:00, 70.9MiB/s]                   
ERROR:datasetsforecast.utils:ERROR, something went wrong downloading data
INFO:datasetsforecast.utils:Successfully downloaded Hourly-test.csv, 132820, bytes.
4.34MiB [00:00, 107MiB/s]                  
ERROR:datasetsforecast.utils:ERROR, something went wrong downloading data
INFO:datasetsforecast.utils:Successfully downloaded M4-info.csv, 4335598, bytes.
100%|██████████| 3.56M/3.56M [00:00<00:00, 57.8MiB/s]
INFO:datasetsforecast.utils:Successfully downloaded submission-Naive2.zip, 3564691, bytes.
INFO:datasetsforecast.utils:Decompressing zip file...
INFO:datasetsforecast.utils:Successfully decompressed /root/m4/datasets/submission-Naive2.zip


Let's take a peak at the dataset:

In [0]:
display(
  spark.sql(f"select * from {catalog}.{db}.m4_hourly_train where unique_id in ('H1', 'H2', 'H3', 'H4', 'H5') order by unique_id, ds")
  )

ds,unique_id,y
2025-01-01T00:00:00Z,H1,585.0
2025-01-01T01:00:00Z,H1,527.0
2025-01-01T02:00:00Z,H1,462.0
2025-01-01T03:00:00Z,H1,437.0
2025-01-01T04:00:00Z,H1,413.0
2025-01-01T05:00:00Z,H1,407.0
2025-01-01T06:00:00Z,H1,404.0
2025-01-01T07:00:00Z,H1,420.0
2025-01-01T08:00:00Z,H1,441.0
2025-01-01T09:00:00Z,H1,471.0


If the number of time series is larger than the number of total cores, we set `spark.sql.shuffle.partitions` to the number of cores (can also be a multiple) so that we don't under-utilize the resource.

In [0]:
if n > sc.defaultParallelism:
    sqlContext.setConf("spark.sql.shuffle.partitions", sc.defaultParallelism)

### Models
Let's configure a list of models we are going to apply to our time series for evaluation and forecasting. A comprehensive list of all supported models is available in [mmf_sa/models/README.md](https://github.com/databricks-industry-solutions/many-model-forecasting/blob/main/mmf_sa/models/README.md). Look for the models where `model_type: local`; these are the local models we import from [statsforecast](https://github.com/Nixtla/statsforecast). Check their documentations for the description of each model. 


In [0]:
active_models = [
    "StatsForecastBaselineWindowAverage",
    "StatsForecastBaselineSeasonalWindowAverage",
    "StatsForecastBaselineNaive",
    "StatsForecastBaselineSeasonalNaive",
    "StatsForecastAutoArima",
    "StatsForecastAutoETS",
    "StatsForecastAutoCES",
    "StatsForecastAutoTheta",
    "StatsForecastAutoTbats",
    "StatsForecastAutoMfles",
    "StatsForecastTSB",
    "StatsForecastADIDA",
    "StatsForecastIMAPA",
    "StatsForecastCrostonClassic",
    "StatsForecastCrostonOptimized",
    "StatsForecastCrostonSBA",
    "SKTimeProphet",
    ]

### Run MMF

Now, we can run the evaluation and forecasting using `run_forecast` function defined in [mmf_sa/models/__init__.py](https://github.com/databricks-industry-solutions/many-model-forecasting/blob/main/mmf_sa/models/__init__.py). Make sure to set `freq="H"` in `run_forecast` function.

In [0]:
run_forecast(
    spark=spark,
    train_data=f"{catalog}.{db}.m4_hourly_train",
    scoring_data=f"{catalog}.{db}.m4_hourly_train",
    scoring_output=f"{catalog}.{db}.hourly_scoring_output",
    evaluation_output=f"{catalog}.{db}.hourly_evaluation_output",
    group_id="unique_id",
    date_col="ds",
    target="y",
    freq="H",
    prediction_length=24,
    backtest_length=168,
    stride=24,
    metric="smape",
    train_predict_ratio=1,
    data_quality_check=True,
    resample=False,
    active_models=active_models,
    experiment_path=f"/Users/{user}/mmf/m4_hourly",
    use_case_name="m4_hourly",
)

Run quality checks


INFO:mmf_sa.data_quality_checks:Starting data quality checks...
INFO:mmf_sa.data_quality_checks:Initial dataset: 298080 records across 414 groups
INFO:mmf_sa.data_quality_checks:Running mandatory configuration checks...
INFO:mmf_sa.data_quality_checks:Running optional data quality checks...
INFO:mmf_sa.data_quality_checks:All groups passed data quality checks
INFO:mmf_sa.data_quality_checks:Data quality checks completed successfully. Final dataset: 298080 records across 414 groups


Finished quality checks
Starting evaluate_score
Starting evaluate_models
Started evaluating StatsForecastBaselineWindowAverage
  metric_name  metric_value
0       smape      0.367656
Finished evaluating StatsForecastBaselineWindowAverage
Started evaluating StatsForecastBaselineSeasonalWindowAverage
  metric_name  metric_value
0       smape      0.334212
Finished evaluating StatsForecastBaselineSeasonalWindowAverage
Started evaluating StatsForecastBaselineNaive
  metric_name  metric_value
0       smape      0.419431
Finished evaluating StatsForecastBaselineNaive
Started evaluating StatsForecastBaselineSeasonalNaive
  metric_name  metric_value
0       smape      0.427644
Finished evaluating StatsForecastBaselineSeasonalNaive
Started evaluating StatsForecastAutoArima
  metric_name  metric_value
0       smape      0.278523
Finished evaluating StatsForecastAutoArima
Started evaluating StatsForecastAutoETS
  metric_name  metric_value
0       smape      0.419106
Finished evaluating StatsForec

'5a6c439e-e6eb-472f-b033-45151ccafc19'

### Evaluate
In `evaluation_output` table, the we store all evaluation results for all backtesting trials from all models.

In [0]:
display(
  spark.sql(f"""
    select * from {catalog}.{db}.hourly_evaluation_output 
    where unique_id = 'H1'
    order by unique_id, model, backtest_window_start_date
    """))

unique_id,backtest_window_start_date,metric_name,metric_value,forecast,actual,model_pickle,run_id,run_date,model,use_case,model_uri
H1,2025-01-24T00:00:00Z,smape,0.0443789220090663,"List(627.9091880323118, 570.5050441521315, 520.1051453264074, 485.28611871024805, 466.98232171788106, 459.5065436191837, 457.26116214218143, 461.05302790636324, 478.07626290680037, 515.3504836022813, 572.3939347597727, 639.5883352518401, 703.5191832413808, 754.4978550037894, 790.0888750065097, 812.7011782901857, 824.7497402799065, 826.3320092588674, 816.8595463477972, 797.6497044515484, 771.7771333298213, 741.0389504033183, 703.6719781852216, 656.1896735756433)","List(587.0, 537.0, 492.0, 464.0, 443.0, 427.0, 424.0, 430.0, 424.0, 473.0, 537.0, 616.0, 684.0, 761.0, 793.0, 826.0, 833.0, 835.0, 838.0, 823.0, 795.0, 750.0, 739.0, 679.0)",gAWVtQABAAAAAACMHHNrdGltZS5mb3JlY2FzdGluZy5mYnByb3BoZXSUjAdQcm9waGV0lJOUKYGUfZQojARmcmVxlIwBSJSMD2FkZF9zZWFzb25hbGl0eZROjBRhZGRfY291bnRyeV9ob2xpZGF5c5ROjAZncm93dGiUjAZsaW4= (truncated),5a6c439e-e6eb-472f-b033-45151ccafc19,2025-07-23T03:45:29.8007Z,SKTimeProphet,m4_hourly,
H1,2025-01-25T00:00:00Z,smape,0.07814723615358801,"List(594.9267066359763, 532.8165746618089, 478.3083266814697, 439.87803066974334, 418.1244997886775, 407.0921619768059, 401.20143252598626, 401.5354840218197, 415.60442253042515, 450.5794823267307, 505.9334971580338, 571.9239676068917, 635.0560019808984, 685.6212390560448, 721.1641207797048, 744.0454489788643, 756.6629761803995, 759.2009910618418, 751.2324181292388, 734.161127156622, 710.9332855926256, 683.0315247459938, 648.4109489020353, 603.5767569294318)","List(622.0, 558.0, 513.0, 476.0, 449.0, 437.0, 422.0, 423.0, 415.0, 475.0, 553.0, 624.0, 680.0, 720.0, 769.0, 805.0, 828.0, 836.0, 849.0, 844.0, 808.0, 757.0, 730.0, 670.0)",gAWVXQoBAAAAAACMHHNrdGltZS5mb3JlY2FzdGluZy5mYnByb3BoZXSUjAdQcm9waGV0lJOUKYGUfZQojARmcmVxlIwBSJSMD2FkZF9zZWFzb25hbGl0eZROjBRhZGRfY291bnRyeV9ob2xpZGF5c5ROjAZncm93dGiUjAZsaW4= (truncated),5a6c439e-e6eb-472f-b033-45151ccafc19,2025-07-23T03:45:29.8007Z,SKTimeProphet,m4_hourly,
H1,2025-01-26T00:00:00Z,smape,0.042956058513496265,"List(581.6468531438629, 523.403186100121, 473.22119105185857, 439.4035675692146, 422.2909617882627, 415.76586826378457, 414.32507361668496, 419.31737475449756, 438.4777986940922, 478.937790911094, 539.8505248695188, 611.0960912985781, 679.0206970776, 734.0700209975464, 774.0900163534947, 801.6121946983114, 818.9188038114461, 825.8742693714628, 821.7511628299995, 807.8392784301645, 787.1511537639924, 761.293766653221, 728.3089056346018, 684.757262099142)","List(594.0, 528.0, 474.0, 447.0, 423.0, 412.0, 413.0, 431.0, 449.0, 489.0, 544.0, 610.0, 696.0, 765.0, 813.0, 851.0, 872.0, 883.0, 899.0, 897.0, 871.0, 831.0, 813.0, 749.0)",gAWVMAABAAAAAACMHHNrdGltZS5mb3JlY2FzdGluZy5mYnByb3BoZXSUjAdQcm9waGV0lJOUKYGUfZQojARmcmVxlIwBSJSMD2FkZF9zZWFzb25hbGl0eZROjBRhZGRfY291bnRyeV9ob2xpZGF5c5ROjAZncm93dGiUjAZsaW4= (truncated),5a6c439e-e6eb-472f-b033-45151ccafc19,2025-07-23T03:45:29.8007Z,SKTimeProphet,m4_hourly,
H1,2025-01-27T00:00:00Z,smape,0.04220238341486709,"List(652.2091244177426, 594.5803193999737, 544.7794567110528, 511.0386456849559, 493.6915997682378, 486.67144217111866, 484.4933979572842, 488.43406905409427, 506.1210155992624, 544.6686398126055, 603.3692727063175, 672.3215807599778, 738.007725059027, 790.82895792498, 828.4695542723076, 853.3505982450433, 867.8098258696257, 871.8835297518822, 864.9657157158547, 848.2942565264176, 824.6984560405589, 795.6425922871651, 759.2108124856704, 712.1794658473683)","List(664.0, 550.0, 544.0, 505.0, 483.0, 469.0, 466.0, 487.0, 492.0, 531.0, 583.0, 659.0, 743.0, 811.0, 863.0, 898.0, 914.0, 920.0, 926.0, 919.0, 887.0, 862.0, 829.0, 769.0)",gAWVGAkBAAAAAACMHHNrdGltZS5mb3JlY2FzdGluZy5mYnByb3BoZXSUjAdQcm9waGV0lJOUKYGUfZQojARmcmVxlIwBSJSMD2FkZF9zZWFzb25hbGl0eZROjBRhZGRfY291bnRyeV9ob2xpZGF5c5ROjAZncm93dGiUjAZsaW4= (truncated),5a6c439e-e6eb-472f-b033-45151ccafc19,2025-07-23T03:45:29.8007Z,SKTimeProphet,m4_hourly,
H1,2025-01-28T00:00:0

### Forecast
In `scoring_output` table, forecasts for each time series from each model are stored.

In [0]:
display(spark.sql(f"""
                  select * from {catalog}.{db}.hourly_scoring_output 
                  where unique_id = 'H1'
                  order by unique_id, model, ds
                  """))

unique_id,ds,y,model_pickle,run_id,run_date,use_case,model,model_uri
H1,"List(2025-01-31T00:00:00Z, 2025-01-31T01:00:00Z, 2025-01-31T02:00:00Z, 2025-01-31T03:00:00Z, 2025-01-31T04:00:00Z, 2025-01-31T05:00:00Z, 2025-01-31T06:00:00Z, 2025-01-31T07:00:00Z, 2025-01-31T08:00:00Z, 2025-01-31T09:00:00Z, 2025-01-31T10:00:00Z, 2025-01-31T11:00:00Z, 2025-01-31T12:00:00Z, 2025-01-31T13:00:00Z, 2025-01-31T14:00:00Z, 2025-01-31T15:00:00Z, 2025-01-31T16:00:00Z, 2025-01-31T17:00:00Z, 2025-01-31T18:00:00Z, 2025-01-31T19:00:00Z, 2025-01-31T20:00:00Z, 2025-01-31T21:00:00Z, 2025-01-31T22:00:00Z, 2025-01-31T23:00:00Z)","List(649.1281972281488, 590.8861961913326, 540.5221311770691, 506.1815027598202, 488.34076254916437, 481.18560525068096, 479.25954153725036, 483.4689502325099, 500.9267060181727, 538.6037888032938, 596.302834723553, 664.9855140757544, 731.6416693971496, 786.3057668829279, 825.577107878177, 850.8216547233285, 864.0889559362199, 866.0269615126617, 857.0042017130264, 838.8820087505875, 814.4549261236581, 784.7548876780505, 747.5176159624799, 699.4889087801458)",gAWVuCwBAAAAAACMHHNrdGltZS5mb3JlY2FzdGluZy5mYnByb3BoZXSUjAdQcm9waGV0lJOUKYGUfZQojARmcmVxlIwBSJSMD2FkZF9zZWFzb25hbGl0eZROjBRhZGRfY291bnRyeV9ob2xpZGF5c5ROjAZncm93dGiUjAZsaW4= (truncated),5a6c439e-e6eb-472f-b033-45151ccafc19,2025-07-23T03:45:29.8007Z,m4_hourly,SKTimeProphet,
H1,"List(2025-01-31T00:00:00Z, 2025-01-31T01:00:00Z, 2025-01-31T02:00:00Z, 2025-01-31T03:00:00Z, 2025-01-31T04:00:00Z, 2025-01-31T05:00:00Z, 2025-01-31T06:00:00Z, 2025-01-31T07:00:00Z, 2025-01-31T08:00:00Z, 2025-01-31T09:00:00Z, 2025-01-31T10:00:00Z, 2025-01-31T11:00:00Z, 2025-01-31T12:00:00Z, 2025-01-31T13:00:00Z, 2025-01-31T14:00:00Z, 2025-01-31T15:00:00Z, 2025-01-31T16:00:00Z, 2025-01-31T17:00:00Z, 2025-01-31T18:00:00Z, 2025-01-31T19:00:00Z, 2025-01-31T20:00:00Z, 2025-01-31T21:00:00Z, 2025-01-31T22:00:00Z, 2025-01-31T23:00:00Z)","List(718.5410985661913, 718.5410985661913, 718.5410985661913, 718.5410985661913, 718.5410985661913, 718.5410985661913, 718.5410985661913, 718.5410985661913, 718.5410985661913, 718.5410985661913, 718.5410985661913, 718.5410985661913, 718.5410985661913, 718.5410985661913, 718.5410985661913, 718.5410985661913, 718.5410985661913, 718.5410985661913, 718.5410985661913, 718.5410985661913, 718.5410985661913, 718.5410985661913, 718.5410985661913, 718.5410985661913)",gAWVWkoAAAAAAACMEnN0YXRzZm9yZWNhc3QuY29yZZSMDVN0YXRzRm9yZWNhc3SUk5QpgZR9lCiMBm1vZGVsc5RdlIwUc3RhdHNmb3JlY2FzdC5tb2RlbHOUjAVBRElEQZSTlCmBlH2UKIwFYWxpYXOUjAVBRElEQZSMFHByZWQ= (truncated),5a6c439e-e6eb-472f-b033-45151ccafc19,2025-07-23T03:45:29.8007Z,m4_hourly,StatsForecastADIDA,
H1,"List(2025-01-31T00:00:00Z, 2025-01-31T01:00:00Z, 2025-01-31T02:00:00Z, 2025-01-31T03:00:00Z, 2025-01-31T04:00:00Z, 2025-01-31T05:00:00Z, 2025-01-31T06:00:00Z, 2025-01-31T07:00:00Z, 2025-01-31T08:00:00Z, 2025-01-31T09:00:00Z, 2025-01-31T10:00:00Z, 2025-01-31T11:00:00Z, 2025-01-31T12:00:00Z, 2025-01-31T13:00:00Z, 2025-01-31T14:00:00Z, 2025-01-31T15:00:00Z, 2025-01-31T16:00:00Z, 2025-01-31T17:00:00Z, 2025-01-31T18:00:00Z, 2025-01-31T19:00:00Z, 2025-01-31T20:00:00Z, 2025-01-31T21:00:00Z, 2025-01-31T22:00:00Z, 2025-01-31T23:00:00Z)","List(659.0, 659.0, 659.0, 659.0, 659.0, 659.0, 659.0, 659.0, 659.0, 659.0, 659.0, 659.0, 659.0, 659.0, 659.0, 659.0, 659.0, 659.0, 659.0, 659.0, 659.0, 659.0, 659.0, 659.0)",gAWVWnQAAAAAAACMEnN0YXRzZm9yZWNhc3QuY29yZZSMDVN0YXRzRm9yZWNhc3SUk5QpgZR9lCiMBm1vZGVsc5RdlIwUc3RhdHNmb3JlY2FzdC5tb2RlbHOUjAlBdXRvQVJJTUGUk5QpgZR9lCiMAWSUTowBRJROjAVtYXhfcJQ= (truncated),5a6c439e-e6eb-472f-b033-45151ccafc19,2025-07-23T03:45:29.8007Z,m4_hourly,StatsForecastAutoArima,
H1,"List(2025-01-31T00:00:00Z, 2025-01-31T01:00:00Z, 2025-01-31T02:00:00Z, 2025-01-31T03:00:00Z, 2025-01-31T04:00:00Z, 2025-01-31T05:00:00Z, 2025-01-31T06:00:00Z, 2025-01-31T07:00:00Z, 2025-01-31T08:00:00Z, 2025-01-31T09:00:00Z, 2025-01-31T10:00:00Z, 2025-01-31T11:00:00Z, 2025-01-31T12:00:00Z, 2025-01-31T13:00:00Z, 2025-01-31T14:00:00Z, 2025-01-31T15:00:00Z, 2025-01-31T16:00:00Z,

Refer to the [notebook](https://github.com/databricks-industry-solutions/many-model-forecasting/blob/main/examples/post-evaluation-analysis.ipynb) for guidance on performing fine-grained model selection after running `run_forecast`.

### Delete Tables
Let's clean up the tables.

In [0]:
#display(spark.sql(f"delete from {catalog}.{db}.hourly_evaluation_output"))

In [0]:
#display(spark.sql(f"delete from {catalog}.{db}.hourly_scoring_output"))